## Reinforcement Learning to Descend a contour

This project is to create a terrain-aware navigation/path planning agent. Goals here are to:
* Utilize reinforcement learning with PyTorch
* Determine a route that would be serviceable for pedestrian travel, whilst lowering the cost of path building

### Setup:
* Convert DEM tiles to 2D grids
* Define an agent that moves on the grid (8 directions)
* Define state: local height patch around the agent, slope, roughness, and land-cover overlays
* Action space: 8 directions (N/S/E/W/NE/NW/SE/SW)
* Reward ideas: Negative reward for steep slopes or high elevation change; Positive reward for reaching a target location with minimal 'effort' (elevation)
* First draft model: CNN-based policy/value network with PPO or DQN for descrete actions

### Converting DEM files to 2D grids:

`ca2023_gaps_dem_*.tif` files are the DEM files (containing LiDAR-derived DEM). `ca2023_gaps_dem_*.tfw` are world files that store:
* Pixel size
* rotation
* upper-left corner coordinates

These files will be fed into Rasterio, then converted into a NumPy array that can be used by PyTorch tensors.

In [ ]:
import rasterio

path = "data\ca2023_gaps_dem_J1317560tR0_C0.tif"

with rasterio.open(path) as src:
    arr = src.read(1) # elevation grid

    print(arr.shape) # get the shape of the given file
    print(src.crs) # Provides UTM zone
    print(src.transform) # Affine transformation matching the given file

(20417, 4419)
EPSG:6339
| 0.50, 0.00, 550970.50|
| 0.00,-0.50, 4125247.50|
| 0.00, 0.00, 1.00|


Notes for myself:
UTM Zone - Universal Transverse Mercator. Divides the Earth into 60 vertical strips, each 6 degrees of longitude wide.
Key points:
* Zones are numbered 1-60 from West t East
* California spans UTM Zone 10N and UTM Zone 11N
* CRS EPSG:6339 is:
    * NAD83(2011) / UTM Zone 10N
    * Measured in meters

Affine transform: a 3x3 matrix that informs rasterio on how to convert:
* Pixel coordinates -> real-world coordinates
* (row, col) -> (x,y)

Now that I've loaded one file, I'll try loading them all.

In [2]:
from pathlib import Path

data_dir = Path("data")
tif_files = sorted(data_dir.glob("*.tif"))

dem_arrays = []

for tif in tif_files:
    with rasterio.open(tif) as src:
        arr = src.read(1)
        dem_arrays.append((tif.name, arr, src.transform, src.crs))

In [3]:
dem_arrays[0]

('ca2023_gaps_dem_J1317560tR0_C0.tif',
 array([[-9.9999900e+05, -9.9999900e+05, -9.9999900e+05, ...,
          8.9633514e+01,  8.9483711e+01,  8.9349220e+01],
        [-9.9999900e+05, -9.9999900e+05, -9.9999900e+05, ...,
          8.9680603e+01,  8.9554489e+01,  8.9407463e+01],
        [-9.9999900e+05, -9.9999900e+05, -9.9999900e+05, ...,
          8.9712723e+01,  8.9578407e+01,  8.9433838e+01],
        ...,
        [-9.9999900e+05, -9.9999900e+05, -9.9999900e+05, ...,
         -9.9999900e+05, -9.9999900e+05, -9.9999900e+05],
        [-9.9999900e+05, -9.9999900e+05, -9.9999900e+05, ...,
         -9.9999900e+05, -9.9999900e+05, -9.9999900e+05],
        [-9.9999900e+05, -9.9999900e+05, -9.9999900e+05, ...,
         -9.9999900e+05, -9.9999900e+05, -9.9999900e+05]],
       shape=(20417, 4419), dtype=float32),
 Affine(0.5, 0.0, 550970.5,
        0.0, -0.5, 4125247.5),
 CRS.from_wkt('PROJCS["NAD83(2011) / UTM zone 10N",GEOGCS["NAD83(2011)",DATUM["NAD83_National_Spatial_Reference_System_2011"

### Time to convert the tiles into PyTorch-ready tensors

This involves normalizing, tiling, and storing.

In [8]:
import torch
import numpy as np

def dem_to_tensor(arr):
    # Replace no-data values if they are present
    arr = np.where(np.isnan(arr), 0, arr) # Specifcally replacing with 0

    # Convert to float32
    arr = arr.astype(np.float32)

    # Min-max normalization
    arr_min = arr.min()
    arr_max = arr.max()
    arr = (arr - arr_min) / (arr_max - arr_min + 1e-10) # Adding small division to avoid divide by 0s

    tensor = torch.from_numpy(arr).unsqueeze(0) # Adding a channel dimension

    return tensor

In [9]:
tensor_tiles = [] # List to hold the resulting tensors

for name, arr, transform, crs in dem_arrays:
    tensor = dem_to_tensor(arr)
    tensor_tiles.append((name, tensor, transform, crs))

### Last steps to turn the `tensor_tiles` into actual square patches for tiling

In [ ]:
def tiling(tensor, tile_size = 512):
    _, H, W = tensor.shape
    tiles = []

    for r in range(0, H - tile_size + 1, tile_size):
        for c in range(0, W - tile_size + 1, tile_size):
            patch = tensor[:, r:r+tile_size, c:c+tile_size]
            tiles.append((patch, r, c))

    return tiles